In [2]:
import numpy as np
import os
import pandas as pd
from io import StringIO
import io
import string
import random
import json
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

In [24]:
df0 = pd.read_csv("modeling data/0_Clean_data.csv")
df00 = pd.read_csv("modeling data/00_Clean_data.csv")

df1 = pd.read_csv("modeling data/1_players_info_added_Clean_data.csv")
df2 = pd.read_csv("modeling data/2_drop_wt.csv")
df3 = pd.read_csv("modeling data/3_fill_rank.csv")
df4 = pd.read_csv("modeling data/4_fill_nationality.csv")
df5 = pd.read_csv("modeling data/5_fill_ht.csv")
df6 = pd.read_csv("modeling data/6_fill_age.csv")
# df7 = pd.read_csv("modeling data/7_drop_outliers.csv")
# df8 = pd.read_csv("modeling data/8_normalized_values.csv")
# df9 = pd.read_csv("modeling data/9_encoded_values.csv")

# print(len(df0))
# print(len(df1))
# print(len(df2))
# print(len(df3))
# print(len(df4))
# print(len(df5))
# print(len(df6))


# Check number of null values in train and test data
df0_nulls = df0.isnull().sum()
df00_nulls = df00.isnull().sum()
df1_nulls = df1.isnull().sum()
df2_nulls = df2.isnull().sum()
df3_nulls = df3.isnull().sum()
df4_nulls = df4.isnull().sum()
df5_nulls = df5.isnull().sum()
df6_nulls = df6.isnull().sum()


print(df0_nulls)



Unnamed: 0           0
Location             0
Date                 0
Round                0
Winner               0
Loser                0
WRank               60
LRank              206
W_hand               0
W_ht              8012
W_nationality     7914
W_age             7915
L_hand               0
L_ht              8275
L_nationality     7978
L_age             7983
L_wt             12433
W_wt             12250
dtype: int64


In [25]:
print(df1_nulls)


Unnamed: 0           0
Location             0
Date                 0
Round                0
Winner               0
Loser                0
WRank               60
LRank              206
W_hand               0
W_ht               957
W_nationality      858
W_age              859
L_hand               0
L_ht              1102
L_nationality      791
L_age              796
L_wt             11818
W_wt             11801
dtype: int64


In [26]:
print(df2_nulls)


Unnamed: 0          0
Location            0
Date                0
Round               0
Winner              0
Loser               0
WRank              60
LRank             206
W_hand              0
W_ht              957
W_nationality     858
W_age             859
L_hand              0
L_ht             1102
L_nationality     791
L_age             796
dtype: int64


In [27]:
print(df3_nulls)


Unnamed: 0          0
Location            0
Date                0
Round               0
Winner              0
Loser               0
WRank               0
LRank               0
W_hand              0
W_ht              957
W_nationality     858
W_age             859
L_hand              0
L_ht             1102
L_nationality     791
L_age             796
dtype: int64


In [28]:
print(df4_nulls)


Unnamed: 0          0
Location            0
Date                0
Round               0
Winner              0
Loser               0
WRank               0
LRank               0
W_hand              0
W_ht              957
W_nationality       0
W_age             859
L_hand              0
L_ht             1102
L_nationality       0
L_age             796
dtype: int64


In [29]:
print(df5_nulls)


Unnamed: 0         0
Location           0
Date               0
Round              0
Winner             0
Loser              0
WRank              0
LRank              0
W_hand             0
W_ht               0
W_nationality      0
W_age            859
L_hand             0
L_ht               0
L_nationality      0
L_age            796
dtype: int64


In [30]:
print(df6_nulls)

Unnamed: 0       0
Location         0
Date             0
Round            0
Winner           0
Loser            0
WRank            0
LRank            0
W_hand           0
W_ht             0
W_nationality    0
W_age            0
L_hand           0
L_ht             0
L_nationality    0
L_age            0
dtype: int64
